## A Comprehensive Guide on various types of `prompting methods` using `Langchain` - `PART1`
* `@author: Snehotosh Banerjee`
* `Date of creation: 03-05-2023`


In [51]:
import os
import openai
import IPython

In [61]:
# Langchain modules
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [53]:
# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_KEY")

In [36]:
def set_open_params(
    model="text-davinci-003", ## model to be used
    temperature=0.7,          ## (controls randomness) Less is more deterministic, more is more random
    max_tokens=256,           ## 
    top_p=1,                  ## the “top p” parameter is like a filter that controls how many different words or phrases 
                              ## the language model considers when it’s trying to predict the next word.
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, prompt):
    """ GET completion from openai api"""

    response = openai.Completion.create(
        engine = params['model'],
        prompt = prompt,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

### Basic Prompt

In [37]:
params = set_open_params()

In [38]:
prompt = "Who is the PM of India"

response = get_completion(params, prompt)
response.choices[0].text
IPython.display.Markdown(response.choices[0].text)



The Prime Minister of India is Narendra Modi.

### Text Summarization Prompt

In [39]:
instruction = "summarize this text in one sentence"
prompt = f"{instruction}: Because of the differences in the way the aircraft fly, there are also differences in what they’re able to do. An airplane is more limited than a helicopter in its abilities and will not be able to do all the things that make a helicopter so cool and useful. For example, an airplane needs a long runway in order to take off while a helicopter can take off by lifting straight up. This means that a helicopter is able to go places that an airplane can’t because it will be able to get out again without a long runway. Helicopters are able to land in many places that would otherwise be inaccessible, which makes them very useful in search and rescue missions."

response = get_completion(params, extra_prompt)
response.choices[0].text
IPython.display.Markdown(response.choices[0].text)



Due to their distinct characteristics, airplanes and helicopters have different capabilities that make helicopters more useful in certain situations such as search and rescue missions.

### Text classification

In [40]:
instruction = "Classify the text into neutral, negative or positive."
prompt = f"{instruction}: I think food was good"

response = get_completion(params, prompt)
response.choices[0].text
IPython.display.Markdown(response.choices[0].text)

, but service was a bit slow

Neutral

### Query Generation

In [41]:
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\""

print(prompt)
response = get_completion(params, prompt)
response.choices[0].text
IPython.display.Markdown(response.choices[0].text)

"""
Table departments, columns = [DepartmentId, DepartmentName]
Table students, columns = [DepartmentId, StudentId, StudentName]
Create a MySQL query for all students in the Computer Science Department
"""



SELECT StudentId, StudentName
FROM students
WHERE DepartmentId = (SELECT DepartmentId FROM departments WHERE DepartmentName = 'Computer Science');

### LLMs and External Tools with Agents

* Used `LLM` models and `Google Search API` as tool
* Used agents

In [42]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [43]:
llm = OpenAI(temperature=0)

In [45]:
tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [46]:
agent.run("Do you know the cause of death of Jock Zonfrillo?")



> Entering new AgentExecutor chain...
 I should research this question.
Action: Search
Action Input: "Jock Zonfrillo cause of death"
Observation: EXCLUSIVE: Inside the secret health battle Jock Zonfrillo has been fighting behind closed doors for years - after the MasterChef star's sudden death at 46 shocked the TV world. MasterChef judge Jock Zonfrillo secretly battled bowel cancer for years, sources have said.
Thought: This article suggests that Jock Zonfrillo's cause of death was bowel cancer.
Final Answer: Jock Zonfrillo's cause of death was bowel cancer.

> Finished chain.


"Jock Zonfrillo's cause of death was bowel cancer."

### Prompt Template

In [54]:
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai.api_key)

In [57]:
instruction = "Respond in one short sentence"
template = f"{instruction}:" + "I really want to travel to {location}. What should I do there?"

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

In [60]:
final_prompt = prompt.format(location='Rome')

print (f"Final Prompt: {final_prompt}")
print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: Respond in one short sentence:I really want to travel to Rome. What should I do there?
LLM Output: 

Explore the city's historical sites, monuments, and museums.


### Chaining

* Simple Chaining
* Using agents
* Add state to chains and agents

#### Simple Chaining

In [62]:
llm = OpenAI(temperature=0.9)

In [63]:
prompt = PromptTemplate(
    input_variables=["food"],
    template="What are 5 vacation destinations for someone who likes to eat {food}?",
)

In [64]:
chain = LLMChain(llm=llm, prompt=prompt)

In [65]:
print(chain.run("dessert"))



1. Paris, France - for the world-famous pastries and desserts, such as éclairs and tarts. 
2. New York City, USA - for classic American desserts like cheesecake and cookies. 
3. Tokyo, Japan - for traditional Japanese sweets such as mochi and warabimochi. 
4. Vienna, Austria - for the famous Sachertorte, a delicious chocolate cake. 
5. Barcelona, Spain - for churros and classic Spanish desserts like Crema Catalana.


#### Memory chaining

In [66]:
from langchain import OpenAI, ConversationChain

In [67]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

In [68]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [69]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

In [70]:
conversation.predict(input="What was the first thing I said to you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: What was the first thing I said to you?
AI:

> Finished chain.


' You said "Hi there!"'

In [71]:
conversation.predict(input="what is an alternative phrase for the first thing I said to you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: What was the first thing I said to you?
AI:  You said "Hi there!"
Human: what is an alternative phrase for the first thing I said to you?
AI:

> Finished chain.


' An alternative phrase for the first thing you said to me could be "Greetings!"'

### Querying Tabular Data

* `SQL` agent
* `Pandas` agent
* `CSV` agent

#### Pandas Agent

In [73]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI
import pandas as pd

In [74]:
df = pd.read_csv('titanic.csv')

In [75]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [76]:
llm = OpenAI(temperature=0) ## Making it utmost deterministic

In [78]:
agent = create_pandas_dataframe_agent(llm, df, verbose=True)

In [79]:
'''
Query
'''
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 891
Thought: I now know the final answer
Final Answer: There are 891 rows.

> Finished chain.


'There are 891 rows.'

In [80]:
'''
Query
'''
agent.run("which gender survived least?")



> Entering new AgentExecutor chain...
Thought: I need to find out the number of survivors for each gender
Action: python_repl_ast
Action Input: df.groupby('Sex')['Survived'].sum()
Observation: Sex
female    233
male      109
Name: Survived, dtype: int64
Thought: I now know the final answer
Final Answer: Male survived least.

> Finished chain.


'Male survived least.'